Primero trasnformamos desde STATA a CSV.

In [1]:
import pandas as pd

# Leer sin convertir categorías
df = pd.read_stata("casen_datos_2017.dta", convert_categoricals=False)

reader = pd.io.stata.StataReader("casen_datos_2017.dta")
value_labels = reader.value_labels()
reader.close()

# Guardar
df.to_csv("CASEN__2017.csv", index=False, encoding='utf-8-sig')
print(f"{len(df)} filas guardadas")

C:\Users\Liz\AppData\Local\Temp\ipykernel_15680\4034272276.py:8: FutureWarning: The StataReader.close() method is not part of the public API and will be removed in a future version without notice. Using StataReader as a context manager is the only supported method.
  reader.close()


216439 filas guardadas


In [2]:
df.head()

,folio,o,id_vivienda,hogar,region,provincia,comuna,zona,expr,expc,...,hh_d_estado,hh_d_servbas,hh_d_entorno,hh_d_accesi,hh_d_medio,hh_d_appart,hh_d_tsocial,hh_d_seg,pobreza_multi_4d,pobreza_multi_5d
0,1.101100e+11,1.0,1.101100e+09,1.0,1.0,11.0,1101.0,1.0,39.0,33.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,NaN,NaN
1,1.101100e+11,1.0,1.101100e+09,1.0,1.0,11.0,1101.0,1.0,39.0,33.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,1.101100e+11,1.0,1.101100e+09,1.0,1.0,11.0,1101.0,1.0,39.0,33.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,NaN,NaN
3,1.101100e+11,2.0,1.101100e+09,1.0,1.0,11.0,1101.0,1.0,39.0,33.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,NaN,NaN
4,1.101100e+11,1.0,1.101100e+09,1.0,1.0,11.0,1101.0,1.0,39.0,33.0,...,1.0,1.0,1.0,0.0,1.0,NaN,1.0,0.0,0.0,NaN


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216439 entries, 0 to 216438
Columns: 810 entries, folio to Sexo
dtypes: float64(768), int64(1), object(41)
memory usage: 1.3+ GB


Ver datos de pobreza.

In [3]:
print('pobreza' in df.columns)
print(df['pobreza'].value_counts())
print('pobreza_multi_5d' in df.columns)
print(df['pobreza_multi_5d'].value_counts())


True
pobreza
3.0    195357
2.0     15260
1.0      5614
Name: count, dtype: int64
True
pobreza_multi_5d
0.0    161600
1.0     44972
Name: count, dtype: int64


In [6]:
import numpy as np

In [16]:

mapeo_texto = {
    1: 'Bajo',
    2: 'Medio', 
    3: 'Alto',
    4: 'Bajo-medio',
    5: 'Bajo-alto',
    6: 'Bajo-medio-alto',
    7: 'Medio-alto'
}

conditions = [
    df['pobreza'] == 1,                                      # 1 - Bajo (Indigentes)
    df['pobreza'] == 2,                                      # 4 - Bajo-medio (Pobres)
    (df['pobreza'] == 3) & (df['ypchautcor'] >= df['ypchautcor'].quantile(0.90)),  # 3 - Alto
    (df['pobreza'] == 3) & (df['ypchautcor'] >= df['ypchautcor'].quantile(0.75)),  # 7 - Medio-alto
    (df['pobreza'] == 3) & (df['ypchautcor'] >= df['ypchautcor'].quantile(0.60)),  # 6 - Bajo-medio-alto
    (df['pobreza'] == 3) & (df['ypchautcor'] >= df['ypchautcor'].quantile(0.40)),  # 2 - Medio
    (df['pobreza'] == 3) & (df['ypchautcor'] >= df['ypchautcor'].quantile(0.25)),  # 5 - Bajo-alto
]

choices = [1, 4, 3, 7, 6, 2, 5]

df['nse_calculado'] = np.select(conditions, choices, default=4)  # Por defecto Bajo-medio
df['Nivel socioeconomico'] = df['nse_calculado'].map(mapeo_texto)

C:\Users\Liz\AppData\Local\Temp\ipykernel_15680\432033060.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['nse_calculado'] = np.select(conditions, choices, default=4)  # Por defecto Bajo-medio
C:\Users\Liz\AppData\Local\Temp\ipykernel_15680\432033060.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Nivel socioeconomico'] = df['nse_calculado'].map(mapeo_texto)


In [33]:
mapeo_sistema_pro = {
    99: 'no sabe',
    1: 'Fonasa', 
    2: 'Fonasa',
    3: 'Fonasa',
    4: 'Fonasa',
    5: 'Fonasa',
    6: 'FF.AA. y del Orden',
    7: 'Isapre',
    8: 'Ninguno (particular)',
    9: 'Otro sistema'
}

# Aplicar el mapeo
df['Sistema Provisional'] = df['s12'].map(mapeo_sistema_pro)

In [18]:
mapeo_pobreza = {
    1: 'Pobreza Extrema',
    2: 'Pobreza No Extrema', 
    3: 'No Pobreza'
}
df['Pobreza'] = df['pobreza'].map(mapeo_pobreza)

C:\Users\Liz\AppData\Local\Temp\ipykernel_15680\3783429643.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Pobreza'] = df['pobreza'].map(mapeo_pobreza)


In [19]:
mapeo_sexo = {
    1: 'Hombre',
    2: 'Mujer'
}

# Aplicar el mapeo
df['Sexo'] = df['sexo'].map(mapeo_sexo)

C:\Users\Liz\AppData\Local\Temp\ipykernel_15680\1727529262.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Sexo'] = df['sexo'].map(mapeo_sexo)


In [26]:
print('fecha_año' in df.columns)

True


In [28]:
df['fecha_año'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 216439 entries, 0 to 216438
Series name: fecha_año
Non-Null Count   Dtype  
--------------   -----  
216439 non-null  float64
dtypes: float64(1)
memory usage: 1.7 MB


In [29]:
df['Año'] = df['fecha_año'].astype(int)
df['Ingreso por Hogar'] = df['ytotcorh']

C:\Users\Liz\AppData\Local\Temp\ipykernel_15680\4173714429.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Año'] = df['fecha_año'].astype(int)
C:\Users\Liz\AppData\Local\Temp\ipykernel_15680\4173714429.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Ingreso por Hogar'] = df['ytotcorh']


In [30]:
df['Año'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 216439 entries, 0 to 216438
Series name: Año
Non-Null Count   Dtype
--------------   -----
216439 non-null  int64
dtypes: int64(1)
memory usage: 1.7 MB


In [36]:
df['Pobreza Multidimensional'] = df['pobreza_multi_5d']

C:\Users\Liz\AppData\Local\Temp\ipykernel_15680\2205339598.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Pobreza Multidimensional'] = df['pobreza_multi_5d']


Creamos el dataframe de pobreza y limpiamos valores nulos

In [38]:
# Variables correctas disponibles
variables_necesarias = ['folio', 'id_vivienda', 'Pobreza', 'Pobreza Multidimensional','Año','Sistema Provisional','comuna',
                        'Nivel socioeconomico', 'region', 'edad', 'Sexo', 'Ingreso por Hogar']

df_pobreza = df[variables_necesarias].copy()

# Eliminar filas donde pobreza sea nula
df_pobreza = df_pobreza.dropna(subset=['Pobreza'])
df_pobreza = df_pobreza.dropna(subset=['Pobreza Multidimensional'])


In [40]:
df_pobreza.head()

,folio,id_vivienda,Pobreza,Pobreza Multidimensional,Año,Sistema Provisional,comuna,Nivel socioeconomico,region,edad,Sexo,Ingreso por Hogar
1,1.101100e+11,1.101100e+09,No Pobreza,1.0,2017,Ninguno (particular),1101.0,Medio,1.0,21.0,Mujer,211091.0
7,1.101100e+11,1.101100e+09,No Pobreza,1.0,2017,Fonasa,1101.0,Bajo-medio-alto,1.0,62.0,Mujer,463333.0
8,1.101100e+11,1.101100e+09,No Pobreza,1.0,2017,Fonasa,1101.0,Bajo-medio-alto,1.0,36.0,Mujer,463333.0
16,1.101100e+11,1.101100e+09,No Pobreza,0.0,2018,Fonasa,1101.0,Medio,1.0,75.0,Hombre,438259.0
17,1.101100e+11,1.101100e+09,No Pobreza,0.0,2017,Fonasa,1101.0,Bajo-medio-alto,1.0,24.0,Mujer,420000.0


In [39]:
df_pobreza.info()

<class 'pandas.core.frame.DataFrame'>
Index: 206572 entries, 1 to 216438
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   folio                     206572 non-null  float64
 1   id_vivienda               206572 non-null  float64
 2   Pobreza                   206572 non-null  object 
 3   Pobreza Multidimensional  206572 non-null  float64
 4   Año                       206572 non-null  int64  
 5   Sistema Provisional       206572 non-null  object 
 6   comuna                    206572 non-null  float64
 7   Nivel socioeconomico      206572 non-null  object 
 8   region                    206572 non-null  float64
 9   edad                      206572 non-null  float64
 10  Sexo                      206572 non-null  object 
 11  Ingreso por Hogar         206572 non-null  float64
dtypes: float64(7), int64(1), object(4)
memory usage: 20.5+ MB


Ahora lo guardamos en CSV.

In [41]:
df_pobreza.to_csv("CASEN_POBREZA_2017.csv", index=False, encoding='utf-8-sig')

Codificamos para la comuna

In [43]:
df_geografico = pd.read_csv("codigos_geograficos_casen2022.csv")

In [45]:
df_final = df_pobreza.merge(
    df_geografico,
    left_on='comuna', 
    right_on='codigo_comuna',
    how='left'
)

In [46]:
df_final.head()

,folio,id_vivienda,Pobreza,Pobreza Multidimensional,Año,Sistema Provisional,comuna,Nivel socioeconomico,region,edad,Sexo,Ingreso por Hogar,codigo_comuna,nombre_comuna,codigo_provincia,nombre_provincia
0,1.101100e+11,1.101100e+09,No Pobreza,1.0,2017,Ninguno (particular),1101.0,Medio,1.0,21.0,Mujer,211091.0,1101,Iquique,11,Iquique
1,1.101100e+11,1.101100e+09,No Pobreza,1.0,2017,Fonasa,1101.0,Bajo-medio-alto,1.0,62.0,Mujer,463333.0,1101,Iquique,11,Iquique
2,1.101100e+11,1.101100e+09,No Pobreza,1.0,2017,Fonasa,1101.0,Bajo-medio-alto,1.0,36.0,Mujer,463333.0,1101,Iquique,11,Iquique
3,1.101100e+11,1.101100e+09,No Pobreza,0.0,2018,Fonasa,1101.0,Medio,1.0,75.0,Hombre,438259.0,1101,Iquique,11,Iquique
4,1.101100e+11,1.101100e+09,No Pobreza,0.0,2017,Fonasa,1101.0,Bajo-medio-alto,1.0,24.0,Mujer,420000.0,1101,Iquique,11,Iquique


In [47]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206572 entries, 0 to 206571
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   folio                     206572 non-null  float64
 1   id_vivienda               206572 non-null  float64
 2   Pobreza                   206572 non-null  object 
 3   Pobreza Multidimensional  206572 non-null  float64
 4   Año                       206572 non-null  int64  
 5   Sistema Provisional       206572 non-null  object 
 6   comuna                    206572 non-null  float64
 7   Nivel socioeconomico      206572 non-null  object 
 8   region                    206572 non-null  float64
 9   edad                      206572 non-null  float64
 10  Sexo                      206572 non-null  object 
 11  Ingreso por Hogar         206572 non-null  float64
 12  codigo_comuna             206572 non-null  int64  
 13  nombre_comuna             206572 non-null  o

In [48]:
df_final = df_final.drop(['codigo_provincia','nombre_provincia','codigo_comuna','comuna'], axis=1)
df_final = df_final.rename(columns={'nombre_comuna': 'Comuna'})
df_final.head()

,folio,id_vivienda,Pobreza,Pobreza Multidimensional,Año,Sistema Provisional,Nivel socioeconomico,region,edad,Sexo,Ingreso por Hogar,Comuna
0,1.101100e+11,1.101100e+09,No Pobreza,1.0,2017,Ninguno (particular),Medio,1.0,21.0,Mujer,211091.0,Iquique
1,1.101100e+11,1.101100e+09,No Pobreza,1.0,2017,Fonasa,Bajo-medio-alto,1.0,62.0,Mujer,463333.0,Iquique
2,1.101100e+11,1.101100e+09,No Pobreza,1.0,2017,Fonasa,Bajo-medio-alto,1.0,36.0,Mujer,463333.0,Iquique
3,1.101100e+11,1.101100e+09,No Pobreza,0.0,2018,Fonasa,Medio,1.0,75.0,Hombre,438259.0,Iquique
4,1.101100e+11,1.101100e+09,No Pobreza,0.0,2017,Fonasa,Bajo-medio-alto,1.0,24.0,Mujer,420000.0,Iquique


Finalmente creamos el dataframe definitivo

In [50]:
df_final.to_csv("CASEN_POBREZA_GEOGRAFICA_2017.csv", index=False, encoding='utf-8-sig')